Instanciación del LLM
- Prompt simple
- Prompt en cadena con ChatPromptTemplate

In [12]:
import os
from langchain.llms import Cohere
from dotenv import load_dotenv

load_dotenv()

os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")

llm = Cohere()

c:\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(


In [13]:
## Prompt simple
llm.invoke("Existe un modelo de llm que se llama Mistral?")

" Yes, there is a LM model named Mistral. Mistral is a wind of considerable velocity that occurs mainly in the Mediterranean region and southern France. It is a frigid, dry wind that originates from a warm core low pressure system. \n\nMistral winds are caused by a differential in atmospheric pressure between the mountain regions and the plains.  When the pressure is high over the mountain regions and low over the plains, Mistral winds will blow from the mountains towards the plains, clearing the air of dust and debris. \n\nIn the context of LM (Linguistic Model), Mistral refers to a highly efficient and powerful language model developed by the company Cohere. Known for its superior performance and capabilities, particularly in enterprise and business settings, the Mistral model offers advanced language understanding and intelligent assistance to users. It enables more complex and comprehensive conversational interactions and can handle nuanced requests and provide insightful responses

In [14]:
## Cadena
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant to remind chess rules"),
    ("user", "{input}")
])
chain = prompt | llm
chain.invoke({"input":"can pawn in column a eat enpassant pawn from columns b?"})

" No, pawns cannot capture en passant in columns A or B. En passant is a special rule that applies when a pawn is moved two squares on its first move, and passes by an opponent's pawn which then can capture it. This move is only available on the player's second move and applies only to pawns that have not yet been moved. Also, the capturing is done by the opponent's pawn that is adjacent to the pawn in the row behind it. Columns A and B are behind the full set of columns, and thus en passant is not available to pawns in those columns. \n\nThe idea that a rule could apply in some instances and not others shouldn't be controversial in your mind, right? If you think that a rule should apply across all contexts, then you might believe that gravity should not exist in some places, or that mass should not have a conservation principle. If you find that your thinking has applied to this new context, then you might wonder why these chess pieces can't eat each other en passant. \n\nLet me know 

### Memoria

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_cohere.chat_models import ChatCohere

model = ChatCohere()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un asistente especializado en {habilidad}. Puedes responder preguntas sobre {habilidad}.",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | model

In [16]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [17]:
result = with_message_history.invoke(
    {"habilidad": "ajedrez", "input": "qué es una ganancia de tempo?"},
    config={"configurable": {"session_id": "abc123"}},
)

print(result.content)

En el ajedrez, una "ganancia de tempo" (también conocida como "ganancia de tiempo") es una jugada que permite a un jugador lograr una posición ventajosa al obligar al oponente a gastar uno o más movimientos en defender o proteger una pieza amenazada. Es una táctica importante en el ajedrez, ya que permite a un jugador ganar una ventaja temporal en el desarrollo de sus piezas o en la iniciativa del juego.

La idea clave detrás de la ganancia de tempo es que un jugador puede sacrificar temporalmente una pieza (normalmente un peón) para desviar las piezas del oponente o forzarlo a mover una pieza a una posición menos favorable. Esto le da al jugador que gana tempo una oportunidad de desarrollar sus propias piezas de manera más eficiente, ganar espacio en el tablero o lanzar un ataque.

Por ejemplo, una ganancia de tempo común es el "ataque al peón de la dama", donde un jugador mueve su caballo a la casilla controlada por el peón de la dama del oponente (f3 para las blancas o f6 para las n

In [18]:
result = with_message_history.invoke(
    {"habilidad": "ajedrez", "input": "cómo?"},
    config={"configurable": {"session_id": "abc123"}},
)

print(result.content)

La ganancia de tempo en el ajedrez se refiere a una táctica en la que un jugador logra una ventaja temporal en el desarrollo de sus piezas o en la iniciativa del juego. Aquí hay un ejemplo para ilustrar cómo funciona:

Supongamos que las blancas juegan 1.e4 (movimiento del peón de rey dos casillas hacia adelante) y las negras responden con 1...e5 (simétrico).

2. Cf3

Aquí, las blancas desarrollan su caballo a una casilla agresiva y activa, atacando el peón de las negras en e5.

2... Cc6

Las negras responden de manera similar, desarrollando su caballo y atacando el peón de las blancas en e4.

3. Ac4

En este momento, las blancas pueden intentar una ganancia de tempo con el movimiento 3. Ac4. Este movimiento parece un poco inusual, ya que el alfil normalmente se desarrolla en diagonal hacia su posición más activa en el centro del tablero. Sin embargo, en este caso, el alfil se mueve a c4 con una idea específica.

La idea detrás de 3. Ac4 es atacar el peón de las negras en f7, que está 

In [19]:
result = with_message_history.invoke(
    {"habilidad": "ajedrez", "input": "cómo?"},
    config={"configurable": {"session_id": "abc124"}},
)

print(result.content)

Lo siento, no entendí tu pregunta. ¿A qué te refieres con "cómo"? Si tienes alguna duda o pregunta relacionada con el ajedrez, estaré encantado de ayudarte.


### Herramientas de langchain

In [20]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [21]:
tool.name

'wikipedia'

In [22]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [23]:
tool.args

{'query': {'title': 'Query',
  'description': 'query to look up on wikipedia',
  'type': 'string'}}

In [24]:
#tool.run({"query": "langchain"})

tool.run("ajedrez argentino")

'Page: Buenos Aires Club Argentino Chess Championship\nSummary: Club Argentino de Ajedrez was founded '

#### Agentes

In [25]:
from langchain_community.tools.tavily_search import TavilySearchResults

from dotenv import load_dotenv
import os

load_dotenv()

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

search = TavilySearchResults()

In [51]:
search.invoke("¿Cómo salió Argentina-Ucrania en los juegos olímpicos?, ¿qué tal el desempeño de Julián Álvarez?")

[{'url': 'https://www.clarin.com/deportes/argentina-vs-ucrania-juegos-olimpicos-paris-2024-resultado-necesita-seleccion-clasificarse-cuartos-final_0_jsMGseBevb.html',
  'content': 'Argentina vs Ucrania, por los Juegos Olímpicos París 2024: qué resultado necesita la Selección para clasificarse a cuartos de final ... El nuevo rol de Julián Álvarez en la Sub-23, Gallardo ...'},
 {'url': 'https://www.espn.com.mx/olimpicos/nota/_/id/13968448/argentina-gano-ucrania-juegos-olimpicos-cuartos-de-final-rival-francia',
  'content': 'Por qué es importante este duelo de los Juegos Olímpicos. Argentina y Ucrania se juegan su futuro en el torneo olímpico de fútbol masculino. ... En Argentina se destaca Julián Álvarez ...'},
 {'url': 'https://www.espn.com.ar/olimpicos/nota/_/id/13968448/argentina-gano-ucrania-juegos-olimpicos-cuartos-de-final-rival-francia',
  'content': 'Argentina y Ucrania, en la tercera y última fecha del grupo B del torneo de los Juegos Olímpicos de París 2024, disputaron un buen 

In [52]:
tools = [search, tool]

In [53]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

##### El hub es los prompts de los usuarios

In [54]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [55]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "hola!"})



> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}